This code is based on the SleepAccel dataset.

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import tensorflow as tf
from importSleepAccel import import_sleep_accel
#%matplotlib ipympl

import argparse
import keras

In [3]:
# Read the ids from a text file
ids_df = pd.read_csv('../Data/SleepAccel/ids.txt', delimiter=",", header=None)


ids = ids_df.iloc[0,:].tolist()
len(ids)

31

In [4]:
accel_epochs = []
sleep_epochs = []
for id in ids:
    l = len(sleep_epochs)
    print(f"Processing id {id}")
    acc_df = pd.read_csv(f'../Data/SleepAccel/motion/{id}_acceleration.txt', delimiter=" ", header=None)
    acc_df = acc_df.to_numpy()
    # Import sleep stage data
    sleep_df = pd.read_csv(f'../Data/SleepAccel/labels/{id}_labeled_sleep.txt', delimiter=" ", header=None).to_numpy()
    epoch_length = 30
    start_time = acc_df[0,0]
    #Round the start time up to the nearest epoch
    start_time = start_time + (epoch_length - (start_time % epoch_length))
    # Get the end time using the earlier end time of accel_t or hr_t
    end_time = acc_df[-1,0]
    # Round the end time down to the nearest epoch
    end_time = end_time - (end_time % epoch_length)

    epoch_times = np.arange(start_time, end_time, epoch_length)
    expected_length = 4.5*60 * 50 # 2.5 minutes of data at 50 Hz

    length = int(4.5*60 * 50 - 50)
    for et in epoch_times:
        start = et - 120 # 2 minutes before the epoch
        end = et + 150 # 2 minutes after the epoch
        accel = acc_df[(acc_df[:,0] >= start) & (acc_df[:,0] <= end)]
        if accel.shape[0] < expected_length - 50:  # If there are less than 99% of the expected samples, skip this epoch
            #print(f"\tSkipping epoch at {et} due to insufficient data: {accel.shape[0]} samples")
            continue
        
        accel_epochs.append(accel[:length,1:4])  # Append only the x, y, z columns
        sleep_epochs.append(sleep_df[sleep_df[:,0] == et, 1])  # Append the sleep stage for this epoch

    print(f"\tExtracted {len(accel_epochs)-l} epochs from id {id}")

Processing id 8000685
	Extracted 974 epochs from id 8000685
Processing id 6220552
	Extracted 1054 epochs from id 6220552
Processing id 8686948
	Extracted 1015 epochs from id 8686948
Processing id 1066528
	Extracted 773 epochs from id 1066528
Processing id 844359
	Extracted 951 epochs from id 844359
Processing id 1818471
	Extracted 1065 epochs from id 1818471
Processing id 3997827
	Extracted 1040 epochs from id 3997827
Processing id 1360686
	Extracted 980 epochs from id 1360686
Processing id 5498603
	Extracted 863 epochs from id 5498603
Processing id 781756
	Extracted 1023 epochs from id 781756
Processing id 2598705
	Extracted 820 epochs from id 2598705
Processing id 2638030
	Extracted 1069 epochs from id 2638030
Processing id 9961348
	Extracted 990 epochs from id 9961348
Processing id 7749105
	Extracted 229 epochs from id 7749105
Processing id 4426783
	Extracted 1014 epochs from id 4426783
Processing id 4314139
	Extracted 1012 epochs from id 4314139
Processing id 9618981
	Extracted 262

In [5]:
len(sleep_epochs)
for i in range(len(sleep_epochs)):
    if sleep_epochs[i].size > 0 and sleep_epochs[i][0] > 1:
        sleep_epochs[i] = 1
    else:
        sleep_epochs[i] = 0

In [ ]:
sleep_epochs = np.array(sleep_epochs)


In [7]:

accel_epochs.shape, sleep_epochs.shape

((25194, 13450, 3), (25194,))

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(accel_epochs.shape[1], accel_epochs.shape[2])),
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_7 (Conv1D)           (None, 13448, 32)         320       
                                                                 
 max_pooling1d_7 (MaxPoolin  (None, 6724, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_8 (Conv1D)           (None, 6722, 64)          6208      
                                                                 
 max_pooling1d_8 (MaxPoolin  (None, 3361, 64)          0         
 g1D)                                                            
                                                                 
 conv1d_9 (Conv1D)           (None, 3359, 32)          6176      
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, 1679, 32)         

In [18]:
train_inputs = accel_epochs[:int(0.8*len(accel_epochs))]
train_labels = sleep_epochs[:int(0.8*len(sleep_epochs))]
test_inputs = accel_epochs[int(0.8*len(accel_epochs)):]
test_labels = sleep_epochs[int(0.8*len(sleep_epochs)):]
steps_per_epoch = len(train_inputs) // 32
history = model.fit(train_inputs, train_labels, batch_size=32, steps_per_epoch=steps_per_epoch, validation_data=(test_inputs, test_labels), verbose=1)

KeyboardInterrupt: 